In [1]:
# mdp = {
#    state: next_state: [reward]
# }

In [7]:
from random import random
import numpy as np

In [154]:
mdp1={}
mdp2={}

In [3]:
string='BBBB........WWWW'

In [176]:
n=4
def find_possible_positions(state, player, n=4):
    # global n
    result=[]
    if player==1:
        char1="W"
        char2="B"
        sign=-1
    else:
        char1="B"
        char2="W"
        sign=1
    for i in range(n*n):
        if state[i]==char1:
            if i+sign*n < 0:
                continue
            if i+sign*n >= 0 and state[i+sign*n]==".":
                s=state
                s=s[:i]+"."+s[i+1:]
                s=s[:i+sign*n]+char1+s[i+sign*n+1:]
                result.append(s)
            if (i+sign*(n-1))//n == (i+sign*n)//n and state[i+sign*(n-1)]==char2:
                s=state
                s=s[:i]+"."+s[i+1:]
                s=s[:i+sign*(n-1)]+char1+s[i+sign*(n-1)+1:]
                result.append(s)
            if (i+sign*(n+1))//n == (i+sign*n)//n and state[i+sign*(n+1)]==char2:
                s=state
                s=s[:i]+"."+s[i+1:]
                s=s[:i+sign*(n+1)]+char1+s[i+sign*(n+1)+1:]
                result.append(s)
    return result

In [150]:
rewards=[]
def game(epsilon=0.25, gamma=0.9):
    global mdp1, mdp2
    current_state="B"*n+"."*((n)*(n-2))+"W"*n
    states=[current_state]
    over=False
    reward=0
    while not over:
        if mdp1.get(current_state) is None:
            result=find_possible_positions(current_state, 1)
            if len(result)==0:
                mdp1[current_state]={}
                reward=0
                break
            mdp1[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp1[current_state]) == 0:
            break
        if random() < epsilon:
            current_state=np.random.choice(list(mdp1[current_state].keys()))
        else:
            current_state=list(mdp1[current_state].keys())[np.argmax([v[0] for v in mdp1[current_state].values()])]
        states.append(current_state)
        for i in range(n):
            if current_state[i]=="W":
                reward=100
                break
        if reward!=0:
            break
        if mdp2.get(current_state) is None:
            result=find_possible_positions(current_state, 2)
            if len(result)==0:
                mdp2[current_state]={}
                reward=0
                break
            mdp2[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp2[current_state]) == 0:
            break
        if random() < epsilon:
            current_state=np.random.choice(list(mdp2[current_state].keys()))
        else:
            current_state=list(mdp2[current_state].keys())[np.argmax([v[0] for v in mdp2[current_state].values()])]
        states.append(current_state)
        for i in range(n*n-n, n*n):
            if current_state[i]=="B":
                reward=-100
                break
        if reward!=0:
            break
    rewards.append(reward)
    length=len(states)
    for i in range(length-1):
        if length%2==i%2:
            mdp1[states[length-2-i]][states[length-1-i]]=((mdp1[states[length-2-i]][states[length-i-1]][0]*mdp1[states[length-2-i]][states[length-1-i]][1]+reward)/(mdp1[states[length-2-i]][states[length-i-1]][1]+1), mdp1[states[length-2-i]][states[length-1-i]][1]+1)
        else:
            mdp2[states[length-2-i]][states[length-1-i]]=((mdp2[states[length-2-i]][states[length-i-1]][0]*mdp2[states[length-2-i]][states[length-1-i]][1]-reward)/(mdp2[states[length-2-i]][states[length-i-1]][1]+1), mdp2[states[length-2-i]][states[length-1-i]][1]+1)
        reward*=gamma


In [158]:
for i in range(100000000):
    game()

In [90]:
def not_practice_game():
    global mdp1, mdp2
    current_state="B"*n+"."*((n)*(n-2))+"W"*n
    reward=0
    while True:
        if mdp1.get(current_state) is None:
            result=find_possible_positions(current_state, 1)
            if len(result)==0:
                mdp1[current_state]={}
                reward=0
                break
            mdp1[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp1[current_state]) == 0:
            break
        current_state=list(mdp1[current_state].keys())[np.argmax([v[0] for v in mdp1[current_state].values()])]
        for i in range(n):
            if current_state[i]=="W":
                reward=100
                break
        if reward!=0:
            break
        if mdp2.get(current_state) is None:
            result=find_possible_positions(current_state, 2)
            if len(result)==0:
                mdp2[current_state]={}
                reward=0
                break
            mdp2[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp2[current_state]) == 0:
            break
        current_state=list(mdp2[current_state].keys())[np.argmax([v[0] for v in mdp2[current_state].values()])]
        for i in range(n*n-n, n*n):
            if current_state[i]=="B":
                reward=-100
                break
        if reward!=0:
            break
    return reward

In [94]:
optimal_white_mdp={}

In [105]:
def optimal_play_by_white():
    global mdp1, mdp2, optimal_white_mdp
    current_state="B"*n+"."*((n)*(n-2))+"W"*n
    reward=0
    while True:
        if optimal_white_mdp.get(current_state) is None:
            if len(mdp1.get(current_state))==0:
                optimal_white_mdp[current_state]={}
                reward=0
                break
            optimal_white_mdp[current_state]=list(mdp1[current_state].keys())[np.argmax([v[0] for v in mdp1[current_state].values()])]
        if len(mdp1[current_state]) == 0:
            break
        current_state=optimal_white_mdp[current_state]
        for i in range(n):
            if current_state[i]=="W":
                reward=100
                break
        if reward!=0:
            break
        if mdp2.get(current_state) is None:
            result=find_possible_positions(current_state, 2)
            if len(result)==0:
                mdp2[current_state]={}
                reward=0
                break
            mdp2[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp2[current_state]) == 0:
            break
        current_state=np.random.choice(list(mdp2[current_state].keys()))
        for i in range(n*n-n, n*n):
            if current_state[i]=="B":
                reward=-100
                break
        if reward!=0:
            break
    return reward

In [128]:
for i in range(10000000):
    optimal_play_by_white()

In [108]:
optimal_black_mdp={}

In [109]:
def optimal_play_by_black():
    global mdp1, mdp2, optimal_black_mdp
    current_state="B"*n+"."*((n)*(n-2))+"W"*n
    reward=0
    while True:
        if mdp1.get(current_state) is None:
            result=find_possible_positions(current_state, 1)
            if len(result)==0:
                mdp1[current_state]={}
                reward=0
                break
            mdp1[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp1[current_state]) == 0:
            break
        current_state=np.random.choice(list(mdp1[current_state].keys()))
        for i in range(n):
            if current_state[i]=="W":
                reward=100
                break
        if reward!=0:
            break
        if optimal_black_mdp.get(current_state) is None:
            if len(mdp2.get(current_state))==0:
                optimal_black_mdp[current_state]={}
                reward=0
                break
            optimal_black_mdp[current_state]=list(mdp2[current_state].keys())[np.argmax([v[0] for v in mdp2[current_state].values()])]
        if len(mdp2[current_state]) == 0:
            break
        current_state=optimal_black_mdp[current_state]
        for i in range(n*n-n, n*n):
            if current_state[i]=="B":
                reward=-100
                break
        if reward!=0:
            break
    return reward

In [130]:
for i in range(100000000):
    optimal_play_by_black()

KeyboardInterrupt: 

In [159]:
len(mdp1)

5717

In [114]:
import json
with open("optimal_white_mdp.json", "w") as f:
    json.dump(optimal_white_mdp, f, indent=4)

In [132]:
with open("optimal_black_mdp_longer.json", "w") as f:
    json.dump(optimal_black_mdp, f, indent=4)

In [116]:
len(mdp1)

3501

In [117]:
len(mdp2)

3010

In [124]:
len(mdp1)

5652

In [125]:
len(mdp2)

5286

In [160]:
len(mdp1)

5717

In [161]:
len(mdp2)

5337

In [236]:
mdp1_5={}
mdp2_5={}

In [237]:
rewards=[]
def n_game(n=5, epsilon=0.05, gamma=0.9):
    global mdp1_5, mdp2_5, rewards
    current_state="B"*n+"."*((n)*(n-2))+"W"*n
    states=[current_state]
    over=False
    reward=0
    while not over:
        if mdp1_5.get(current_state) is None:
            result=find_possible_positions(current_state, 1, n=n)
            if len(result)==0:
                mdp1_5[current_state]={}
                reward=0
                break
            mdp1_5[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp1_5[current_state]) == 0:
            break
        if random() < epsilon:
            current_state=np.random.choice(list(mdp1_5[current_state].keys()))
        else:
            current_state=list(mdp1_5[current_state].keys())[np.argmax([v[0] for v in mdp1_5[current_state].values()])]
        states.append(current_state)
        for i in range(n):
            if current_state[i]=="W":
                reward=100
                break
        if reward!=0:
            break
        if mdp2_5.get(current_state) is None:
            result=find_possible_positions(current_state, 2, n=n)
            if len(result)==0:
                mdp2_5[current_state]={}
                reward=0
                break
            mdp2_5[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp2_5[current_state]) == 0:
            break
        if random() < epsilon:
            current_state=np.random.choice(list(mdp2_5[current_state].keys()))
        else:
            current_state=list(mdp2_5[current_state].keys())[np.argmax([v[0] for v in mdp2_5[current_state].values()])]
        states.append(current_state)
        for i in range(n*n-n, n*n):
            if current_state[i]=="B":
                reward=-100
                break
        if reward!=0:
            break
    rewards.append(reward)
    if (len(rewards))>100:
        rewards=rewards[-100:]
    length=len(states)
    for i in range(length-1):
        if length%2==i%2:
            mdp1_5[states[length-2-i]][states[length-1-i]]=((mdp1_5[states[length-2-i]][states[length-i-1]][0]*mdp1_5[states[length-2-i]][states[length-1-i]][1]+reward)/(mdp1_5[states[length-2-i]][states[length-i-1]][1]+1), mdp1_5[states[length-2-i]][states[length-1-i]][1]+1)
        else:
            mdp2_5[states[length-2-i]][states[length-1-i]]=((mdp2_5[states[length-2-i]][states[length-i-1]][0]*mdp2_5[states[length-2-i]][states[length-1-i]][1]-reward)/(mdp2_5[states[length-2-i]][states[length-i-1]][1]+1), mdp2_5[states[length-2-i]][states[length-1-i]][1]+1)
        reward*=gamma


In [241]:
for i in range(1000000000):
    n_game()

KeyboardInterrupt: 

In [242]:
print(rewards[-100:])

[0, 0, 0, 0, 0, 0, -100, 0, 100, 100, 0, 100, 0, 100, 100, 0, -100, -100, -100, 0, 0, 0, 0, 0, 0, 100, -100, 0, 0, 0, -100, 0, 100, 0, -100, 0, 0, -100, -100, -100, 0, 0, 0, -100, 0, -100, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, -100, 0, 100, 0, 0, 0, 0, 0, -100, 0, 0, 100, 0, 100, 0, 0, -100, 0, -100, 0, -100, 100, 100, 0, 100, 100, 0, -100, 0, 0, 100, 0, 0, 100, 0, -100, -100, -100, 100, 100, -100, 0, 0, 0]


In [230]:
def n_not_practice_game(n=5):
    global mdp1_5, mdp2_5
    current_state="B"*n+"."*((n)*(n-2))+"W"*n
    reward=0
    while True:
        if mdp1_5.get(current_state) is None:
            result=find_possible_positions(current_state, 1, n=5)
            if len(result)==0:
                mdp1_5[current_state]={}
                reward=0
                break
            mdp1_5[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp1_5[current_state]) == 0:
            break
        current_state=list(mdp1_5[current_state].keys())[np.argmax([v[0] for v in mdp1_5[current_state].values()])]
        print(current_state)
        for i in range(n):
            if current_state[i]=="W":
                reward=100
                break
        if reward!=0:
            break
        if mdp2_5.get(current_state) is None:
            result=find_possible_positions(current_state, 2, n=5)
            if len(result)==0:
                mdp2_5[current_state]={}
                reward=0
                break
            mdp2_5[current_state]={possible_state: (0,1) for possible_state in result}
        if len(mdp2_5[current_state]) == 0:
            break
        current_state=list(mdp2_5[current_state].keys())[np.argmax([v[0] for v in mdp2_5[current_state].values()])]
        print(current_state)
        for i in range(n*n-n, n*n):
            if current_state[i]=="B":
                reward=-100
                break
        if reward!=0:
            break
    print(reward)

In [243]:
for i in range(1):
    n_not_practice_game()

BBBBB............W..WW.WW
BB.BB..B.........W..WW.WW
BB.BB..B....W.......WW.WW
BB..B..BB...W.......WW.WW
BB..B..BW...........WW.WW
BB.....BB...........WW.WW
BB.....BB..........WWW.W.
B.....BBB..........WWW.W.
B.....BBB.........WWWW...
B.....B.B...B.....WWWW...
B.....B.B...W......WWW...
B.......B...B......WWW...
B.......B...B..W...W.W...
.....B..B...B..W...W.W...
.....B..B.W.B......W.W...
.....B....W.BB.....W.W...
.....B....W.BW.......W...
.....B....W..W...B...W...
.....B....W..W...W.......
0


In [247]:
with open("mdp1_5.json", "r") as f:
    mdp1_orig=json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [233]:
mdp2_5['BBBBB............W..WW.WW']

{'.BBBBB...........W..WW.WW': (-4.455019315474058, 60118478),
 'B.BBB.B..........W..WW.WW': (-4.455019362039358, 201677352),
 'BB.BB..B.........W..WW.WW': (-4.76834447134536, 13507488),
 'BBB.B...B........W..WW.WW': (-4.455021295934208, 15412175),
 'BBBB.....B.......W..WW.WW': (-4.455019465750489, 15169551)}

In [245]:
len(mdp1_5)

303304

In [223]:
for key, value in mdp1_5.items():
    for k, v in value.items():
        if v[0]>20:
            print(key, k, v)
    break

In [244]:
len(mdp2_5)

254855

In [1]:
with open("mdp1_5.json", "w") as f:
    json.dump(mdp1_5, f, indent=4)

NameError: name 'json' is not defined

In [235]:
with open("mdp2_5.json", "w") as f:
    json.dump(mdp2_5, f, indent=4)